In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tensorflow.keras.applications.resnet50 import ResNet50


In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

In [ ]:
!unzip -q tiny-imagenet-200.zip && ls tiny-imagenet-200

In [ ]:
def mapDirectoryToInt():
    ID_DICT = {}
    PATH = 'tiny-imagenet-200/wnids.txt'
    FILEOPEN = open(PATH,'r')
    for i, folder in enumerate(FILEOPEN):
        ID_DICT[folder.replace('\n','')]=i
    return ID_DICT

In [ ]:
def getImage(folderName=None):
    DATASET_PATH = 'tiny-imagenet-200/'+folderName+'/'
    directories = os.listdir(DATASET_PATH)
    ONE_HOT_ENCODED = np.eye(len(directories))
    MAP_DIC = mapDirectoryToInt()
    TRAIN_IMAGES = []
    TRAIN_LABELS = []
    CLASS = None
    for directory in directories:
        path = DATASET_PATH+directory+'/'
        files = os.listdir(path)
        if files[0].endswith('.txt'):
            CLASS = files[0]
        else:
            CLASS = files[1]
        for file in files:
            if file.endswith('.txt'):
                continue
            IMG_FOLDER_PATH = path+file+'/'
            IMAGES = os.listdir(IMG_FOLDER_PATH)
            for IMAGE in IMAGES:
                IMG_PATH = IMG_FOLDER_PATH+IMAGE
                img = cv2.imread(IMG_PATH)
                if len(img.shape)< 3:
                    continue
                #img = np.reshape(img,[64,64,3])
                TRAIN_IMAGES.append(img)
                TRAIN_LABELS.append((np.reshape(ONE_HOT_ENCODED[MAP_DIC[CLASS.replace('_boxes.txt','')]], [200])))
    return np.array(TRAIN_IMAGES), np.array(TRAIN_LABELS)
            

In [ ]:
def loadValidationData(folderName=None):
    DATASET_PATH = 'tiny-imagenet-200/'
    DATASET_FILE_PATH = 'tiny-imagenet-200/'+folderName+'/val_annotations.txt'
    ONE_HOT_ENCODED = np.eye(200)
    MAP_DIC = mapDirectoryToInt()
    file = open(DATASET_FILE_PATH)
    TEST = []
    TEST_LABELS = []
    for line in file:
        image, class_id = line.split('\t')[:2]
        IMG_PATH = DATASET_PATH+folderName+'/images/'+image
        img = cv2.imread(IMG_PATH)
        if len(img.shape) <3:
            continue
        TEST.append(img)
        TEST_LABELS.append((np.reshape(ONE_HOT_ENCODED[MAP_DIC[class_id]], [200])))
    return np.array(TEST), np.array(TEST_LABELS)

In [ ]:
from tensorflow import keras
TEST_IMAGES, TEST_LABELS = loadValidationData('val')
TEST_IMAGES = keras.applications.resnet50.preprocess_input(TEST_IMAGES)

In [ ]:
print(TEST_IMAGES.shape)
print(TEST_LABELS.shape)

In [ ]:
TRAIN_IMAGES, TRAIN_LABELS = getImage('train')

In [ ]:

TRAIN_IMAGES =  keras.applications.resnet50.preprocess_input(TRAIN_IMAGES)


In [ ]:
path = 'tiny-imagenet-200/'

In [ ]:
import tensorflow.keras as K

In [ ]:
input_shape = K.Input(shape=(64,64,3))

In [ ]:

resModel = ResNet50(include_top=False, weights='imagenet',input_tensor=input_shape)
#model.summary()

In [ ]:

for layer in resModel.layers[:-2]:
  layer.trainable = False

In [ ]:
import tensorflow as tf 
model = K.models.Sequential()
model.add(resModel)
model.add(K.layers.Flatten())
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dropout(0.7))

model.add(K.layers.Dense(200, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
                  optimizer=K.optimizers.Adam(learning_rate=0.0001),
                  metrics=['accuracy'])

In [ ]:
TRAIN_IMAGES.shape, TRAIN_LABELS.shape, TEST_IMAGES.shape, TEST_LABELS.shape

In [ ]:
history = model.fit(TRAIN_IMAGES, TRAIN_LABELS, batch_size=32,  epochs=50, verbose=1,
                        validation_data=(TEST_IMAGES, TEST_LABELS),shuffle=True)

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title("Model Accuracy(Fine-Tunining)")
plt.show()

In [ ]:
plt.plot(history.history['loss'],'r', label='Training Loss')
plt.plot(history.history['val_loss'],'c', label = 'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title("Model Loss(Fine-Tunining)")
plt.show()

In [ ]:
print('Training accuracy: ', np.around(history.history['accuracy'][49]*100, decimals=2),'%')
print('Validaiton accuracy: ', np.around(history.history['val_accuracy'][49]*100, decimals=2),'%')

In [ ]:
prediction = model.evaluate(TEST_IMAGES,TEST_LABELS)
print("Testing Accuracy: ", np.around(prediction[1]*100, decimals=2))
print("Testing Loss: ", np.around(prediction[0],decimals=2))